# Coffea processor


In [ ]:
from coffea import hist, util

import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import numpy as np
import uproot

from pprint import pprint

import matplotlib.pyplot as plt

The code below introduces some basic concepts for writing code using Coffea.

There are three primary pieces to the Coffea code:

The processor, which contains all of the analysis cuts and fills the histogram in the process function.
The second cell defines the files we want to run over and then runs the code using run_uproot_job.
After we run the processor, we can then plot any of the histograms we have generated.
To test any changes you make to the histograms, you will have to rerun each of the three cells below.



In [ ]:
class MuonSelector(processor.ProcessorABC):
    def __init__(self):
        # In the initializer, any of the outputs you would like to produce are defined (ex. histograms)

        # Coffea histograms are defined in the same way as in the previous exercise
        # define a list of axes first

        #Declare an axis for the dataset
        dataset_axis = hist.Cat("dataset","Dataset")
        
        #Declare an axis for the muon pt
        pt_axis = hist.Bin("pt","$p_{T}$ [GeV]", 40, 0, 200)
        eta_axis = hist.Bin("eta","$\eta$", 50, -2.5, 2.5)
        phi_axis = hist.Bin("phi","$\phi$", 64, -3.2, 3.2)
        
        #Define the accumulator object, a dictionary storing all of the histograms and counters 
        #that we will fill later in the process function
        self._accumulator = processor.dict_accumulator({
            'muon_pt': hist.Hist("Counts", dataset_axis, pt_axis),

        }
        )

    @property
    def accumulator(self):
        return self._accumulator

    

    # The process method is where the heart of the analysis is.  
    # This is where all of the selections are done and the histograms get filled 
    #  (things you did in notebook cells before will be done here instead)
    def process(self, events):
        ### The process function is where most of the work happens. As we'll see below, this is
        ### where the main analysis work happens (object cuts, event selections, filling histograms). 
        
        ## This gets us the accumulator dictionary we defined in init
        output = self.accumulator.identity()

        ## To access variables from the ntuples, use the "events" object
        ## The dataset name is part of events.metadata
        dataset = events.metadata['dataset']

        ## The coffea NanoEventSchema packages all muon variables (columns) into the events.Muon object
        ## Each variable can be accessed using muons.key_name
        muons = events.Muon        
        
        ######
        # Select muons with pt >30, eta < 2.4, tight ID, and relIso < 0.15
        muonSelectTight = ((muons.pt>30) &
                           (abs(muons.eta)<2.4) &
                           (muons.tightId) &
                           (muons.pfRelIso04_all < 0.15)
                          )

        # Apply the selection to muons using the array[mask] syntax. 
        # tightMuons only includes the muons that pass the tight selection we defined
        tightMuons = muons[muonSelectTight]
        
        jets = events.Jet
        
        ######
        # Select jets with pt >30, eta < 2.4, and tight ID
        jetSelectTight = ((jets.pt>30) &
                          (abs(jets.eta)<2.4) &
                          (jets.isTight)
                         )
        tightJets = jets[jetSelectTight]

        ######
        # Select b-tagged jets
        bjetSelectTight = (jetSelectTight &
                           (jets.btagDeepB>0.6321)
                          )
        tightBJets = jets[bjetSelectTight]
        
        
        #Apply a High Level Trigger (HLT) requirement
        #  look for events which passed a single muon trigger, either IsoMu24 or IsoTkMu24
        #  This is a separate branch in the tree, and is stored in NanoEvents under an HLT object
        
        trigger = events.HLT.IsoMu24 | events.HLT.IsoTkMu24
        
        # Select events passing the trigger, with exactly on tight muon, ≥4 jets, and ≥ 1 b-tagged jets. 
        eventSelection = (trigger &
                          (ak.num(tightMuons)==1) &
                          (ak.num(tightJets)>=4) & 
                          (ak.num(tightBJets)>=1))

        # Fill the muon_pt histogram using the tightMuons in events that pass our selection 
        # Note that ak.flatten() is required when filling a histogram to remove the jaggedness
        output['muon_pt'].fill(dataset=dataset,
                              pt=ak.flatten(tightMuons[eventSelection].pt))

        
        return output

    def postprocess(self, accumulator):
        return accumulator

The processor takes a dictionary of files to run on, with a key of the dataset name and a list of files as the items

In [ ]:
#Define files to run over
skimDir="/udrive/staff/dnoonan/Skims"

fileset = {"TTGamma":[f"{skimDir}/TTGamma_SingleLept_2016_skim.root"],
           "TTbar":[f"{skimDir}/TTbarPowheg_Semilept_2016_skim_1of10.root",
                    f"{skimDir}/TTbarPowheg_Semilept_2016_skim_2of10.root"],
           "WGamma":[f"{skimDir}/WGamma_2016_skim.root"],
           "Z+jets":[f'{skimDir}/DYjetsM50_ext2_2016_skim_1of10.root'],
           "W+3jets":[f"{skimDir}/W3jets_2016_skim.root"],
           "W+4jets":[f"{skimDir}/W4jets_2016_skim.root"],
          }

filesetData = {"DataMu":[f"{skimDir}/Data_SingleMu_b_2016_skim_1of10.root"],
              }


This next cell will run over all of the datasets you have selected.  It may take a few minutes to run, but this is because you are running over close to 20 million events of MC

In [ ]:
np.warnings.filterwarnings('ignore')

#the NanoAODSchema needs to be adjusted, to remove cross references to FSRPhotons
class SkimmedSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        super().__init__(base_form)

#Run Coffea code using uproot
outputMC = processor.run_uproot_job(
    fileset,  #dictionary of datasets to run on, defined earlier in this cell
    "Events", #Name of the TTree you will be opening
    MuonSelector(),  #Coffea processor you defined
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 4},  ## workers = 2, parallelize jobs, running 2 at once
    chunksize=1000000, #in each chunk, use 1 million events
#     maxchunks=3, #limit to using only 3 chunks for each dataset (useful for testing purposes)
)



The coffea histograms that are defined in the accumulator in the initializer

ex:
```
self._accumulator = processor.dict_accumulator({
      'muon_pt': hist.Hist("Counts", dataset_axis, pt_axis),
...
```

are what get returned when you run the processor.  In this case, they are stored as `outputMC`

In [ ]:
print(outputMC)

In [ ]:
hist.plot1d(outputMC['muon_pt'],overlay='dataset',stack=True)

Next, we'll run over the actualy data to et something to compare against, and store the histogram outputs as `outputData`

In [ ]:
#Run coffea processor again, this time using the filesetData list
outputData = processor.run_uproot_job(
    filesetData,
    "Events",
    MuonSelector(),
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 4}, 
    chunksize=1000000,
)


Now, we can draw both the histograms from Monte-Carlo (saved in the outputMC) and the histogram of data (outputData) together on the same plot.

This can be done by calling hist.plot1d twice within the same cell, which will make it draw on the same axes.

We also add some different drawing options for the data histogram.  This makes it draw the data as black points, rather than filled histograms

In [ ]:
hist.plot1d(outputMC['muon_pt'],overlay='dataset',stack=True)

data_err_opts = {
    'linestyle':'none',
    'marker': '.',
    'markersize': 10.,
    'color':'k',
    'elinewidth': 1,
}

hist.plot1d(outputData['muon_pt'],overlay='dataset',error_opts=data_err_opts)

Notice that they do not agree at all, this is because we have not yet scaled the Monte-Carlo histograms to match the luminosity of the data

### Histogram Scaling

When comparing a Monte-Carlo to Data, we need to scale the MC to the number of events we expect to see in a given amount of data.

$\text{N}_\text{expected} = \sigma \cdot L$

 - $\text{N}_\text{expected}$ = Number of events expected
 - $\sigma$ = cross section of a specific process
 - $L$ = integrated luminosity of data
 
In MC, we often generate far more events than we expect (for better statistical uncertainties), so we need rescale the MC distributions.  This is done by reweighting each MC dataset, where the weight applied is the ratio of the number of events expected to the number of events produced in the MC ($\text{N}_{MC}$)

$w = \frac{\text{N}_\text{expected}}{\text{N}_\text{MC}} = \frac{\sigma \cdot L}{\text{N}_\text{MC}}$

The number of events in MC and the cross section will change for each dataset

#### Cross sections
| Process | Cross Section (pb) |
| :--- | :---: |
| TTGamma (single lepton) | 7.509 |
| TTbar (single lepton) | 380.095 |
| WGamma | 489 |
| Z+jets | 6077.22 |
| W+3jets | 1165.8108 |
| W+4jets | 592.9176 |

#### Number of events

The $\text{N}_\text{MC}$ value should be the total number of 
Normally, in NanoAOD, you could keep track of the number of events are in the files that you process (tallying the total number of events in each sample, across all chunks processed).

However, since we are running on skims of the full MC sets, some of the events have already been removed.  However, in this case, we get the 


In [ ]:
#the code below uses uproot to open a histogram in the root file, used to track the total
#  number of events processed while producing the skim

nEvents = {}
for d in fileset:
    if not d in nEvents:
        nEvents[d] = 0
    for fName in fileset[d]:
        with uproot.open(fName)['hEvents'] as hEvents:
            nEvents[d] += hEvents.values()[0] + hEvents.values()[2]
pprint(nEvents)

You should get something like:
```
{'TTGamma': 11005200.0,
 'TTbar': 17673700.0,
 'W+3jets': 19798117.0,
 'W+4jets': 9116657.0,
 'WGamma': 6103817.0,
 'Z+jets': 8920292.0}

``` 

### Calculate weights

Make a dictionary, containing the weights to apply for each dataset in fileset
The dictionary should have the same key names as are in fileset, since these are what get used as the 'dataset' in the histogram axis.

The actual CMS data you are using corresponds to an integrated luminosity $L = 578 \text{pb}^{-1}$ (10% of data collected in 2016 Run B, or a little less than 2% of the whole 2016 dataset)

In [ ]:
###############
## To Do
## Make new dictionary named weights, containing the luminosity and cross section based weights for each sample
###############



In [ ]:
cx = {'TTGamma':7.509,
     'TTbar': 380.095,
     'WGamma':489,
     'Z+jets':6077.22,
     'W+3jets':1165.8108,
     'W+4jets':592.9176}
lumi_weight = {}
for keyName in fileset:
    lumi_weight[keyName] = (cx[keyName]*578.)/nEvents[keyName]


In [ ]:
pprint(lumi_weight)

You should get something like:
```
{'TTGamma': 0.00039437738523607027,
 'TTbar': 0.012430612152520412,
 'W+3jets': 0.034035491476285346,
 'W+4jets': 0.037591232487961326,
 'WGamma': 0.04630577882659326,
 'Z+jets': 0.39378006459878223}
```

In [ ]:
#loop over objects in the output, and scale them to the by the weight dictionary you just created
for key, obj in outputMC.items():
    if isinstance(obj, hist.Hist):
        obj.scale(lumi_weight, axis="dataset")

In [ ]:
hist.plot1d(outputData['muon_pt'],overlay='dataset',error_opts=data_err_opts,overflow='over')
hist.plot1d(outputMC['muon_pt'],overlay='dataset',stack=True,overflow='over')


Should look something like:
    
<img src="plots/processor_example_muonPt.png" align="left">

### To do

Add histograms into the coffea processor for the following variables:
 - muon eta
 - muon phi
 - leading jet pt
 - leading jet eta
 - leading jet phi
 - second leading jet pt
 - second leading jet eta
 - second leading jet phi
        

Examples of what the outputs should probably look like:

Muon Kinematics

<table><tr>
<td> <img src="plots/processor_example_muonPt.png" style="width: 300px;"/> </td>    
<td> <img src="plots/processor_example_muonEta.png" style="width: 300px;"/> </td>    
<td> <img src="plots/processor_example_muonPhi.png" style="width: 300px;"/> </td>    
</tr></table>

Leading Jet Kinematics

<table><tr>
<td> <img src="plots/processor_example_leadingJetPt.png" style="width: 300px;"/> </td>    
<td> <img src="plots/processor_example_leadingJetEta.png" style="width: 300px;"/> </td>    
<td> <img src="plots/processor_example_leadingJetPhi.png" style="width: 300px;"/> </td>    
</tr></table>

Second Jet Kinematics

<table><tr>
<td> <img src="plots/processor_example_secondJetPt.png" style="width: 300px;"/> </td>    
<td> <img src="plots/processor_example_secondJetEta.png" style="width: 300px;"/> </td>    
<td> <img src="plots/processor_example_secondJetPhi.png" style="width: 300px;"/> </td>    
</tr></table>


## Next Steps

### Z-boson selector

In this step, we're going to try to find events from a Z boson.

You are going to implement a selection looking for events with exactly two muons that have opposite charge.  These muons should pass the same 'tight' selections used in the previous notebook for selection.

We are looking for events that have exactly two two muons and where the two leptons have opposite charge (one muon and one antimuon).

Then, make the following plots:
 - $p_T$ of the leading muon in the event
 - Mass of the combination of the two leptons
 - Difference between the two leptons in eta
 - Difference between the two leptons in phi
 - Difference between the two leptons in R


In [ ]:
class Zselector(processor.ProcessorABC):
    def __init__(self):
        ### This function is where the histograms are defined and any other initialization happens
        
        ### Muon pt
        #Declare an axis for the dataset
        dataset_axis = hist.Cat("dataset","Dataset")
        
        #Declare an axis for the muon pt

        
        #Define the accumulator object, a dictionary storing all of the histograms and counters 
        #that we will fill later in the process function
        self._accumulator = processor.dict_accumulator({

        }
        )

        
        
    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        ### The process function is where most of the work happens. As we'll see below, this is
        ### where the main analysis work happens (object cuts, event selections, filling histograms). 
        
        ## This gets us the accumulator dictionary we defined in init
        output = self.accumulator.identity()

        ## To access variables from the ntuples, use the "events" object
        ## The dataset name is part of events.metadata
        dataset = events.metadata['dataset']


        
        
        
        
        
        
        
        
        
        
        return output

    def postprocess(self, accumulator):
        return accumulator

In [ ]:
#the NanoAODSchema needs to be adjusted, to remove cross references to FSRPhotons
class SkimmedSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        super().__init__(base_form)

#Run Coffea code using uproot
outputMC_Z = processor.run_uproot_job(
    fileset,  #dictionary of datasets to run on, defined earlier in this cell
    "Events", #Name of the TTree you will be opening
    Zselector(),  #Coffea processor you defined
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 4},  ## workers = 2, parallelize jobs, running 2 at once
    chunksize=1000000, #in each chunk, use 1 million events
#     maxchunks=1, #limit to using only 3 chunks for each dataset (useful for testing purposes)
)

#Scale histograms
for key, obj in outputMC_Z.items():
    if isinstance(obj, hist.Hist):
        obj.scale(lumi_weight, axis="dataset")


In [ ]:
outputData_Z = processor.run_uproot_job(
    filesetData, 
    "Events", 
    Zselector(), 
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 4},  
    chunksize=1000000, 
)


In [ ]:
#Draw plots